# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install geoviews


Note: you may need to restart the kernel to use updated packages.


In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import hvplot.pandas
import requests
#import geoapify
import gmaps
import geoviews
import os
import json
%matplotlib inline
import geopandas

# Import API key
from api_keys import geoapify_key
#gmaps.configure(api_key = geoapify_key)

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [10]:
geoapify_key

'f82b5e7d5df74a3bae35878884224236'

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talcahuano,-36.7167,-73.1167,46.08,100,58,0.00,CL,1690424259
1,1,port-aux-francais,-49.3500,70.2167,39.99,78,90,36.06,TF,1690424269
2,2,port alfred,-33.5906,26.8910,61.41,82,92,10.67,ZA,1690424269
3,3,saint-leu,-21.1500,55.2833,65.26,76,47,11.68,RE,1690424269
4,4,midleton,51.9153,-8.1805,61.12,97,100,11.10,IE,1690424269


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriTerrain",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data

city_name = []

for index, row in city_data_df.iterrows():
    if row["Cloudiness"] == 0:
        if row ["Wind Speed"] < 10:
            if row["Max Temp"] >70 and row["Max Temp"]<80.6:#temperature in fahranite
                city_name.append(row)

city_weather_df = pd.DataFrame(city_name)

city_weather_df.to_csv("city_perfect_weather.csv")
city_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,kelso,46.1468,-122.9084,80.56,46,0,8.05,US,1690424273
54,54,aktau,43.6500,51.2000,72.18,73,0,7.70,KZ,1690424276
74,74,bethel,41.3712,-73.4140,77.31,78,0,8.05,US,1690424029
139,139,humanes de madrid,40.2504,-3.8306,76.24,37,0,1.01,ES,1690424287
145,145,kone,-21.0595,164.8658,76.01,39,0,9.80,NC,1690424287
155,155,tizi-n-tleta,36.5457,4.0571,73.13,50,0,5.10,DZ,1690424289
192,192,zonguldak,41.2500,31.8333,70.92,40,0,6.06,TR,1690424295
261,261,kharabali,47.4196,47.2568,75.81,47,0,6.06,RU,1690424305
291,291,zaragoza,41.6561,-0.8773,70.81,55,0,6.91,ES,1690424034
302,302,westport,41.1415,-73.3579,79.02,78,0,6.91,US,1690424311


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="

input = "Hotel"
radius = "10000"

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,36,kelso,46.1468,-122.9084,80.56,46,0,8.05,US,1690424273,
54,54,aktau,43.6500,51.2000,72.18,73,0,7.70,KZ,1690424276,
74,74,bethel,41.3712,-73.4140,77.31,78,0,8.05,US,1690424029,
139,139,humanes de madrid,40.2504,-3.8306,76.24,37,0,1.01,ES,1690424287,
145,145,kone,-21.0595,164.8658,76.01,39,0,9.80,NC,1690424287,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "radius": 10000,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    # name_address =  (json.dumps(response, indent=4, sort_keys=True))

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Retrieving Results for Index 36: kelso.
kelso - nearest hotel: Best Western Aladdin Inn
Retrieving Results for Index 54: aktau.
aktau - nearest hotel: Гульдана
Retrieving Results for Index 74: bethel.
bethel - nearest hotel: Hampton Inn Danbury
Retrieving Results for Index 139: humanes de madrid.
humanes de madrid - nearest hotel: Hotel Pizarro
Retrieving Results for Index 145: kone.
kone - nearest hotel: Pacifik Appartels
Retrieving Results for Index 155: tizi-n-tleta.
tizi-n-tleta - nearest hotel: Arcade
Retrieving Results for Index 192: zonguldak.
zonguldak - nearest hotel: No hotel found
Retrieving Results for Index 261: kharabali.
kharabali - nearest hotel: No hotel found
Retrieving Results for Index 291: zaragoza.
zaragoza - nearest hotel: Hotel Pilar Plaza
Retrieving Results for Index 302: westport.
westport - nearest hotel: Norwalk Inn and Conference Center
Retrieving Results for Index 307: itapuranga.
itapuranga - nearest hotel: No hotel found
Retrieving 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,36,kelso,46.1468,-122.9084,80.56,46,0,8.05,US,1690424273,Best Western Aladdin Inn
54,54,aktau,43.6500,51.2000,72.18,73,0,7.70,KZ,1690424276,Гульдана
74,74,bethel,41.3712,-73.4140,77.31,78,0,8.05,US,1690424029,Hampton Inn Danbury
139,139,humanes de madrid,40.2504,-3.8306,76.24,37,0,1.01,ES,1690424287,Hotel Pizarro
145,145,kone,-21.0595,164.8658,76.01,39,0,9.80,NC,1690424287,Pacifik Appartels
155,155,tizi-n-tleta,36.5457,4.0571,73.13,50,0,5.10,DZ,1690424289,Arcade
192,192,zonguldak,41.2500,31.8333,70.92,40,0,6.06,TR,1690424295,No hotel found
261,261,kharabali,47.4196,47.2568,75.81,47,0,6.06,RU,1690424305,No hotel found
291,291,zaragoza,41.6561,-0.8773,70.81,55,0,6.91,ES,1690424034,Hotel Pilar Plaza
302,302,westport,41.1415,-73.3579,79.02,78,0,6.91,US,1690424311,Norwalk Inn and Conference Center


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
# Configure the map plot_3
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name","Country"],
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)